Import the relevant python modules required for the example: xarray and matplotlib

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from SubRoutines.indexing import find_matching_index
from SubRoutines.plotting import pannel_plot, pannel_hist

Read in the dataset

In [ ]:
esacci_lst_025 = "Data/ESACCI-LST-L3C-LST-MODISA-0.25deg_1MONTHLY_DAY-20060701000000-fv3.00.nc"
ds = xr.load_dataset(esacci_lst_025)

Display the contents of the dataset

In [ ]:
ds

We can plot the total uncertainty in LST across the globe.  The largest uncertainties can be seen in regions where data coverage is limited due to cloud cover, including the Saharan Desert and across India.  Total cloud cover can be seasonal so the areas of largest uncertainty may vary.

In [ ]:
ds.lst_uncertainty.plot(size=8,aspect=2,robust=True)

Next we can plot a smaller region of the globe to look at the uncertainties in more detail.  Zooming in on this region over China and Japan we can see that some surface features appear in the uncertainty data.  This is because there is a component of the uncertainty that is related to our representation of the land cover.  There are also larger uncertainties along the coast, particularly evident around the islands where there are fewer satellite observations over land within these grid boxes (some are neighbouring ocean pixels) so there are less data going into the LST estimate.  The panel plots below show the different uncertainty components that form the total uncertainty.  The surface features such as rivers and high ground are evident in the bottom right plot.  The larger uncertainties along the coast are split between the surface, atmosphere and random components (note that the colourbar scales are different for each component).  Random uncertainties include instrument noise and sampling - these are uncertainties that are uncorrelated between neighbouring LST values.  Try changing the region by editing the code above so that you can see how the uncertainties vary in the part of the world you are most interested in.

In [ ]:
lower_lon = 105.0
upper_lon = 145.0
lower_lat = 25.0
upper_lat = 65.0
lat_max_idx = find_matching_index(ds.lat.values,upper_lat)
lat_min_idx = find_matching_index(ds.lat.values,lower_lat)
lon_max_idx = find_matching_index(ds.lon.values,upper_lon)
lon_min_idx = find_matching_index(ds.lon.values,lower_lon)
lat_region = slice(lat_min_idx, lat_max_idx)
lon_region = slice(lon_min_idx,lon_max_idx)

In [ ]:
ds.lst_uncertainty.isel(lat=lat_region, lon=lon_region).plot(size=8,aspect=2, robust=True)

In [ ]:

pannel_plot(ds,lat_region,lon_region)

We can plot a histogram of the total uncertainties over this region and alhtough there are one or two larger values, we find that the uncertainties typically range between 0.5-1 K.

In [ ]:
ds.lst_uncertainty.isel(lat=lat_region, lon=lon_region).plot.hist(size=8,bins=50, alpha=0.5, color='steelblue',edgecolor='grey')
;

Similarly we can plot histograms of the different uncertainty components.  In this particular region the largest uncertainties are arising from the surface component.

In [ ]:
pannel_hist(ds,lat_region,lon_region)

To see the different uncertainty contributions and how they propogate we will manually calculate the total uncertainty from the components. First we assign a point to do the calculation for:

In [ ]:
lat_point, lon_point = 35.0, 115.0

lat_idx = find_matching_index(ds.lat.values, lat_point)
lon_idx = find_matching_index(ds.lon.values, lon_point)

Then we extract the total uncertainty and the uncertainty components for that point:

In [ ]:
point_total_uncertainty = ds.lst_uncertainty.isel(lat=lat_idx,lon=lon_idx).values[0]

ran = ds.lst_unc_ran.isel(lat=lat_idx,lon=lon_idx).values[0]
atm = ds.lst_unc_loc_atm.isel(lat=lat_idx,lon=lon_idx).values[0]
sfc = ds.lst_unc_loc_sfc.isel(lat=lat_idx,lon=lon_idx).values[0]
sys = ds.lst_unc_sys.isel(lat=lat_idx,lon=lon_idx).values[0]
print(f"Total: {point_total_uncertainty:.3f}"[:-1])
print(f"random: {ran:.3f}, atmosphere: {atm:.3f}, surface: {sfc:.3f}, systematic: {sys:3f}")

To combine the components to make the total uncertainty they are added in quadrature:

In [ ]:
calculated_total = np.sqrt(ran**2 + atm**2 + sfc**2 + sys**2)
print(f"Calculated total: {calculated_total:.3f}"[:-1])

The uncertainty can be used to give context to the LST in terms of the degree to which we have confidence in the LST value. We can visualise this on a LST transect:

In [ ]:
lat_transect = 30.0
lower_lon = 5.0
upper_lon = 9.0

lat_idx = find_matching_index(ds.lat.values, lat_transect)
lon_max_idx = find_matching_index(ds.lon.values,upper_lon)
lon_min_idx = find_matching_index(ds.lon.values,lower_lon)
lon_segment = slice(lon_min_idx,lon_max_idx)
transect = ds.isel(lat=lat_idx,lon=lon_segment).squeeze()

Now we can plot the LST with a region bounded by the uncertainty shown:

In [ ]:
upper_bound = transect.lst.values+transect.lst_uncertainty.values
lower_bound = transect.lst.values-transect.lst_uncertainty.values

fig, ax = plt.subplots(figsize=(16,8))
transect.lst.plot(ax=ax)
ax.fill_between(transect.lon.values,upper_bound,lower_bound,alpha=0.2)
